### Import the pacakges

In [1]:
from fastai.vision.all import * 
import tifffile as tiff
from natsort import natsorted

### Define the source directory for the raw data

In [2]:
# Replace with your raw data folder 
src = '/home/susa03/post_project/post_dataset/synthetic_data/raw_data'

### Define the destination directory and then create two subfolders for images and masks to store

In [3]:
dst = '/home/susa03/post_project/post_dataset/synthetic_data/sorted_data/sphere'
bse = 'bse'   
masks = 'masks'     ### Store ground truth image
os.makedirs(os.path.join(dst,bse), exist_ok=True)  # create destination folder if not exists
os.makedirs(os.path.join(dst,masks), exist_ok=True)  # create destination folder if not exists

# Get the full path of the created directory
bse_path = os.path.join(dst, bse)
print("BSE path:", bse_path)

masks_path = os.path.join(dst, masks)
print("Mask path:", masks_path)


BSE path: /home/susa03/post_project/post_dataset/synthetic_data/sorted_data/sphere/bse
Mask path: /home/susa03/post_project/post_dataset/synthetic_data/sorted_data/sphere/masks


### Assess the image files and define the image height and width accordingly

In [4]:
image_files = get_image_files(src, recurse=True)
file_count = len(image_files)
digit_count = len(str(file_count))
image_height = 850
image_width = 850

### Define the fuctions to assess each images category wise and then store the bse images in the bse subfolder and the respective mask in the mask folder

In [5]:
def return_cat(filename:str,category:str):
    value = str(category) in str(filename)
    return value
   
def make_category_list(filenames):
    # Perform a natural sort on the image files by name
    sorted_image_files = natsorted(filenames, key=lambda x: x.name)
    # Iterate over the images in groups of 8
    group_size = 8
   
    for i in range(0, len(sorted_image_files), group_size):
        image_group = sorted_image_files[i:i+group_size]
        bse_image= np.zeros((image_height, image_width))  
        current_mask = np.zeros((image_height, image_width)) 
    
        # Process the current group of images
        for file in image_group:
            if return_cat(file,category='_bse'):
                bse_image_address = os.path.join(src,file)
                bse_image = tiff.imread(bse_image_address)
                bse_store_path = bse_path/Path(file.stem + file.suffix)
                #print(bse_store_path)
                tiff.imwrite(bse_store_path,np.array(bse_image,dtype=np.uint8))
            elif return_cat(file,category='_gt'):
                current_mask_address = os.path.join(src,file)
                current_mask = tiff.imread(current_mask_address)
                mask_store_path = masks_path/Path(file.stem + file.suffix)
                #print(mask_store_path)
                tiff.imwrite(mask_store_path,np.array(current_mask,dtype=np.uint8))
            else: pass

make_category_list(image_files)